In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

True

In [2]:
from accounts import Account

In [3]:
account = Account.get("deepu")
account

Account(name='deepu', balance=10000.0, strategy='You are a day trader that aggressively buys and sells shares based on news and market conditions.', holdings={}, transactions=[], portfolio_value_time_series=[('2025-12-02 09:06:26', 10000.0), ('2025-12-02 09:06:30', 10000.0)])

In [4]:
account.buy_shares("AMZN", 3, "Because this bookstore website looks promising")

'Completed. Latest details:\n{"name": "deepu", "balance": 9288.7804, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 237.07319999999999, "timestamp": "2025-12-02 09:21:35", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-12-02 09:06:26", 10000.0], ["2025-12-02 09:06:30", 10000.0], ["2025-12-02 09:21:35", 9998.580399999999]], "total_portfolio_value": 9998.580399999999, "total_profit_loss": -1.4196000000010827}'

In [5]:
account.report()

'{"name": "deepu", "balance": 9288.7804, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 237.07319999999999, "timestamp": "2025-12-02 09:21:35", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-12-02 09:06:26", 10000.0], ["2025-12-02 09:06:30", 10000.0], ["2025-12-02 09:21:35", 9998.580399999999], ["2025-12-02 09:21:37", 9998.580399999999]], "total_portfolio_value": 9998.580399999999, "total_profit_loss": -1.4196000000010827}'

In [6]:
account.list_transactions()

[{'symbol': 'AMZN',
  'quantity': 3,
  'price': 237.07319999999999,
  'timestamp': '2025-12-02 09:21:35',
  'rationale': 'Because this bookstore website looks promising'}]

In [7]:
# Now let's use our accounts server as an MCP server

params = {"command": "uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()


In [8]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is deepu and my account is under the name deepu. What's my balance and my holdings?"
model = "gpt-4.1-mini"

In [9]:

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="account_manager", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))


Deepu, your account balance is $9,288.78. Your current holdings include 3 shares of AMZN (Amazon). Is there anything specific you would like to do with your account?

In [10]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools()
print(mcp_tools)
openai_tools = await get_accounts_tools_openai()
print(openai_tools)

[Tool(name='get_balance', title=None, description='Get the cash balance of the given account name.\n\nArgs:\n    name: The name of the account holder\n', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': 'get_balanceOutput', 'type': 'object'}, icons=None, annotations=None, meta=None), Tool(name='get_holdings', title=None, description='Get the holdings of the given account name.\n\nArgs:\n    name: The name of the account holder\n', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, outputSchema={'additionalProperties': {'type': 'integer'}, 'title': 'get_holdingsDictOutput', 'type': 'object'}, icons=None, annotations=None, meta=None), Tool(name='buy_shares', title=None, description="B

In [11]:
request = "My name is deepu and my account is under the name deepu. What's my balance?"

with trace("account_mcp_client"):
    agent = Agent(name="account_manager", instructions=instructions, model=model, tools=openai_tools)
    result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Your current account balance is $9,288.78. If you have any other questions or need assistance with your account, feel free to ask!

In [12]:
context = await read_accounts_resource("deepu")
print(context)

{"name": "deepu", "balance": 9288.7804, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 237.07319999999999, "timestamp": "2025-12-02 09:21:35", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-12-02 09:06:26", 10000.0], ["2025-12-02 09:06:30", 10000.0], ["2025-12-02 09:21:35", 9998.580399999999], ["2025-12-02 09:21:37", 9998.580399999999], ["2025-12-02 09:22:01", 9998.250399999999]], "total_portfolio_value": 9998.250399999999, "total_profit_loss": -1.74960000000101}


In [13]:
from accounts import Account
Account.get("deepu").report()

'{"name": "deepu", "balance": 9288.7804, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 237.07319999999999, "timestamp": "2025-12-02 09:21:35", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-12-02 09:06:26", 10000.0], ["2025-12-02 09:06:30", 10000.0], ["2025-12-02 09:21:35", 9998.580399999999], ["2025-12-02 09:21:37", 9998.580399999999], ["2025-12-02 09:22:01", 9998.250399999999], ["2025-12-02 09:22:03", 9998.250399999999]], "total_portfolio_value": 9998.250399999999, "total_profit_loss": -1.74960000000101}'